# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

In [4]:
import  sqlite3 # Подключаем модуль 
import pandas as pd
import csv

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [5]:
con = sqlite3.connect("data/recipes.db")
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [30]:
try: 
    cur.execute(
                '''
                CREATE TABLE Recipe
                    (
                        id INTEGER PRIMARY KEY NOT NULL,
                        name TEXT,
                        minutes INTEGER,
                        submitted TEXT,
                        description TEXT,
                        n_ingredients INTEGER
                    );
                '''
                )

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() 
    #cur.close()
    #con.close()


Запрос успешно выполнен


3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [42]:
#cur.execute('DROP TABLE Review')
#con.commit()

In [43]:
try: 
    cur.execute(
                '''
                CREATE TABLE Review
                    (
                        id INTEGER PRIMARY KEY NOT NULL,
                        user_id INTEGER NOT NULL,
                        recipe_id INTEGER,
                        date TEXT,
                        rating REAL,
                        review TEXT,
                        FOREIGN KEY (recipe_id) REFERENCES Recipe (id) 
                    );
                '''
                )

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() 
    #cur.close()
    #con.close()


Запрос успешно выполнен


4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [6]:
with open('data/reviews_sample.csv') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    reviews = []
    for row in csvreader:
        reviews.append(tuple(row))
#reviews

In [44]:
try: 
    cur.executemany('INSERT INTO Review VALUES (?, ?, ?, ?, ?, ?)', reviews)

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() 
    #cur.close()
    #con.close()


Запрос успешно выполнен


In [7]:
with open('data/recipes_sample_with_tags_ingredients.csv') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    recipes = []
    for row in csvreader:
        recipes.append((row[2], row[1], row[3], row[5], row[7], row[8]))
#recipes

In [38]:
try: 
    cur.executemany('INSERT INTO Recipe VALUES (?, ?, ?, ?, ?, ?)', recipes)

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print ("Запрос успешно выполнен") 
    con.commit() 
    #cur.close()
    #con.close()


Запрос успешно выполнен


5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [8]:
cur.execute('''
                SELECT * FROM RECIPE 
                WHERE n_ingredients = 10
                
            ''')

In [9]:
for row in cur.fetchmany(5):
    print(row)

(176, 'cola cake', 55, '1999-08-22', '', 10)
(246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10)
(289, 'feijoada  brazilian bean soup  ii', 150, '1999-10-04', 'brazilian bean soup', 10)
(373, 'tiramisu   balducci s in new york city', 95, '1999-08-17', 'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.', 10)
(544, 'greek mushroom salad', 38, '1999-08-10', '', 10)


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [10]:
cur.execute('''
            SELECT name, MAX(minutes) FROM Recipe      
            ''')
print(cur.fetchone())

('strawberry liqueur', 129615)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [24]:
inp_id = int(input())
try: 
    cur.execute('''
                        SELECT * FROM Recipe
                        WHERE id = (?)
    
                ''', (inp_id,))

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    res = cur.fetchone()
    if res == None:
        print('Рецепт отсутствует')
    else:
        print(res)

-1
Рецепт отсутствует


8. Найдите кол-во отзывов с рейтингом 5.

In [35]:
try: 
    cur.execute('''
                        SELECT * FROM Review
                        WHERE rating = 5
    
                ''')

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    print(len(cur.fetchall()))

91361


9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [40]:
# 1 вариант
try: 
    cur.execute('''
                        SELECT recipe_id, MAX(rating), review FROM Review
                        WHERE rating >= 4
                        GROUP BY recipe_id
                ''')

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    res = cur.fetchall()
    print(len(res))
    #for i in range(0, 10):
      #  print(res[i])

26318


In [44]:
# 2 вариант 
try: 
    cur.execute('''
                        SELECT DISTINCT recipe_id FROM Review
                        WHERE rating >= 4
                ''')

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    res = cur.fetchall()
    print(len(res))
    #for i in range(0, 10):
      #  print(res[i])

26318


10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [50]:
# 2 вариант 
try: 
    cur.execute('''
                        SELECT * FROM Recipe
                        WHERE submitted > '2009-12-31' and submitted < '2011-01-01' 
                        and minutes >= 15
                ''')

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    res = cur.fetchall()
    print(len(res))

1319


11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [ ]:
# 2 вариант 
try: 
    cur.execute('''
                        SELECT Recipe.id, Recipe.name, Review.user_id, Review.date, Review.rating
                        FROM Recipe
                        LEFT JOIN Review ON Recipe.id = Review.recipe_id
                        WHERE Recipe.n_ingredients >= 3
                        ORDER BY Recipe.id
                        
                ''')

except sqlite3.DatabaseError as  err: 
    print("Ошибка:", err) 
else: 
    res = cur.fetchall()
    for row in res:
        print(row)